## FABlib API References Examples

- [fablib.show_config](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.show_config)
- [fablib.list_sites](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_sites)
- [fablib.list_hosts](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_hosts)
- [fablib.new_slice](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.new_slice)
- [slice.add_node](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.add_node)
- [slice.submit](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.submit)
- [slice.get_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.get_nodes)
- [slice.list_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.list_nodesß)
- [slice.show](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.show)
- [node.execute](https://fabric-fablib.readthedocs.io/en/latest/node.html#fabrictestbed_extensions.fablib.node.Node.execute)
- [slice.delete](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.delete) 

In [ ]:
import datetime

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

In [ ]:
slice_name = 'kubernetes-cluster'
image = "default_ubuntu_24"

# Host of the site used for the nodes (Use specific host to avoid error occurred without it: "Timeout waiting for the server to come up")
# TODO: Make random again
site = "AMST"
site_host = "amst-w1.fabric-testbed.net"

node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

node1_nic_name = 'NIC1'
node2_nic_name = 'NIC2'
node3_nic_name = 'NIC3'
network_name = 'NET1'

In [ ]:
# Create a slice
slice = fablib.new_slice(name=slice_name)

# Add Nodes with the specific variables
# Also validate the node can be created and raise an exception in case of failure
node1 = slice.add_node(name=node1_name, site=site, disk=20, validate=True, raise_exception=True, host=site_host, image=image)
node2 = slice.add_node(name=node2_name, site=site, disk=20, validate=True, raise_exception=True, host=site_host, image=image)
node3 = slice.add_node(name=node3_name, site=site, disk=20, validate=True, raise_exception=True, host=site_host, image=image)

interface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
interface2 = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
interface3 = node3.add_component(model='NIC_Basic', name=node3_nic_name).get_interfaces()[0]

net1 = slice.add_l3network(name=network_name, interfaces=[interface1, interface2, interface3], type='IPv4')

# Calculate the lease end time for 2 weeks from now with timezone information
lease_end_time = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(weeks=2)

# Submit the slice, using an end date 2 weeks from now (the current maximum lease time) 
# to make sure that the slice can be used for a longer period of time. Progress shows an indicator of the current progression.
# Wait until the state is finished and use an interval (it may take some time before the slice and nodes are created)
slice.submit(wait=True, wait_timeout=3600, wait_interval=20, progress=True, wait_jupyter='text', lease_end_time=lease_end_time);

In [ ]:
for node in slice.get_nodes():
    stdout, stderr = node.execute('echo Hello, FABRIC from node `hostname -s`')

In [ ]:
network = slice.get_networks()[0];
subnet = network.get_subnet();
gateway = network.get_gateway();
available_ips = network.get_available_ips(10);

In [ ]:
def assign_ip(node):
    interface = node.get_interface(network_name="NET1")
    address = available_ips.pop(0)
    network.allocate_ip(address)
    interface.ip_addr_add(addr=address, subnet=subnet)
    node.ip_route_add(subnet=subnet, gateway=gateway)
    return address

ips = [assign_ip(node) for node in nodes];

print(ips);

In [ ]:
def setup_node(node):
    node.upload_file(local_file_path="node_scripts/node_setup.sh", remote_file_path="setup.sh")
    node.execute(f"chmod +x setup.sh && ./setup.sh");

for node in nodes:
    setup_node(node)

    # Uncomment this if you are going to work on the node itself and want to use k9s
    # node.upload_file(local_file_path="node_scripts/install_k9s.sh", remote_file_path="k9s.sh")
    # node.execute(f"chmod +x k9s.sh && ./k9s.sh");

In [ ]:
inventory = (
    f"[kube_control_plane]\n"
    f"node1 ansible_host={ips[0]} ip={ips[0]} etcd_member_name=etcd1\n"
    f"\n"
    f"[etcd:children]\n"
    f"kube_control_plane\n"
    f"\n"
    f"[kube_node]\n"
)

for i, ip in enumerate(ips[1:]):
    inventory += f"node{i + 2} ansible_host={ip} ip={ip}\n"

with open('kubespray/inventory.ini', 'w') as f:
    f.write(inventory)

In [ ]:
nodes[0].upload_file(local_file_path="node_scripts/control_kubespray_setup.sh", remote_file_path="kubespray_setup.sh");
nodes[0].execute("chmod +x kubespray_setup.sh && ./kubespray_setup.sh");

nodes[0].upload_file(local_file_path="kubespray/inventory.ini", remote_file_path="kubespray/inventory/dynamos/inventory.ini");
nodes[0].upload_file(local_file_path="kubespray/ansible.cfg", remote_file_path="kubespray/ansible.cfg");
nodes[0].upload_file(local_file_path="node_scripts/dot_kube.sh", remote_file_path="dot_kube.sh");
nodes[0].execute("chmod +x ./dot_kube.sh");
nodes[0].upload_file(local_file_path="/home/fabric/work/fabric_config/slice_key", remote_file_path="/home/ubuntu/.ssh/slice_key");
nodes[0].execute("chmod 600 /home/ubuntu/.ssh/slice_key");

In [ ]:
nodes[0].upload_file(local_file_path="node_scripts/start_kubespray.sh", remote_file_path="start.sh");
nodes[0].execute(f"chmod +x start.sh && ./start.sh");

In [ ]:
nodes[0].upload_file(local_file_path="node_scripts/reset_kubespray.sh", remote_file_path="reset.sh");
nodes[0].execute(f"chmod +x reset.sh && ./reset.sh");

In [ ]:
slice.delete()